In [1]:
import pandas as pd
import re
from sqlalchemy import create_engine, inspect
import os
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv()

True

In [2]:


class PostgresManager:
    def __init__(self, host, port, dbname, user, password):
        self.db_params = {
            'host': host,
            'port': port,
            'dbname': dbname,
            'user': user,
            'password': password
        }
        self.engine_str = (
            f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
        )
        self.engine = create_engine(self.engine_str)
        print("✅ PostgreSQL connection initialized.")

    def upload_dataframe(self, df: pd.DataFrame, table_name: str, if_exists='replace'):
        """
        Uploads a DataFrame to PostgreSQL.
        - if_exists: 'replace', 'append', or 'fail'
        """
        try:
            df.to_sql(table_name, self.engine, if_exists=if_exists, index=False, method='multi')
            print(f"✅ Data uploaded to table '{table_name}'.")
        except Exception as e:
            print(f"❌ Failed to upload to '{table_name}': {e}")
    
    def getTicker30MinData(self, ticker: str) -> pd.DataFrame:
        """
        Retrieves 30-minute interval data for a given ticker from PostgreSQL
        """
        table_name = f"{ticker.upper()}_30MinData"
        try:
            query = f"SELECT * FROM \"{table_name}\""
            df = pd.read_sql_query(query, self.engine)
            
            # Convert 'date' column to datetime and set as index
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
                df.set_index('date', inplace=True)
                df.sort_index(inplace=True)
                print(f"✅ Retrieved {len(df)} rows of 30-min data for {ticker}")
            else:
                print(f"❌ 'date' column missing in table {table_name}")
            return df
        except Exception as e:
            print(f"❌ Failed to fetch data for {ticker}: {e}")
            return pd.DataFrame()
    def getTickerEODData(self, ticker: str) -> pd.DataFrame:
        """
        Retrieves EOD data for a given ticker from PostgreSQL
        """
        table_name = f"{ticker.upper()}_EOD_Data"
        try:
            query = f"SELECT * FROM \"{table_name}\""
            df = pd.read_sql_query(query, self.engine)
            
            # Convert 'date' column to datetime and set as index
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
                df.set_index('date', inplace=True)
                df.sort_index(inplace=True)
                print(f"✅ Retrieved {len(df)} rows of EOD data for {ticker}")
            else:
                print(f"❌ 'date' column missing in table {table_name}")
            return df
        except Exception as e:
            print(f"❌ Failed to fetch data for {ticker}: {e}")
            return pd.DataFrame()
    
    def getTickerFundamentalsData(self, ticker: str) -> pd.DataFrame:
        """
        Retrieves fundamentals data for a given ticker from PostgreSQL
        """
        table_name = f"{ticker.upper()}_FundamentalsData"
        try:
            query = f"SELECT * FROM \"{table_name}\""
            df = pd.read_sql_query(query, self.engine)
            
            # Convert 'date' column to datetime and set as index
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
                df.set_index('date', inplace=True)
                df.sort_index(inplace=True)
                print(f"✅ Retrieved {len(df)} rows of fundamentals data for {ticker}")
            else:
                print(f"❌ 'date' column missing in table {table_name}")
            return df
        except Exception as e:
            print(f"❌ Failed to fetch data for {ticker}: {e}")
            return pd.DataFrame()
    def getTickerCombinedData(self, ticker: str) -> pd.DataFrame:
        """
        Retrieves combined data for a given ticker from PostgreSQL
        """
        table_name = f"{ticker.upper()}_CombinedData"
        try:
            query = f"SELECT * FROM \"{table_name}\""
            df = pd.read_sql_query(query, self.engine)
            
            # Convert 'date' column to datetime and set as index
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
                df.set_index('date', inplace=True)
                df.sort_index(inplace=True)
                print(f"✅ Retrieved {len(df)} rows of combined data for {ticker}")
            else:
                print(f"❌ 'date' column missing in table {table_name}")
            return df
        except Exception as e:
            print(f"❌ Failed to fetch data for {ticker}: {e}")
            return pd.DataFrame()
        
    def get_tickers_from_30min_tables(self):
        """
        Extracts all tickers from tables that match the {ticker}_30MinData format.
        """
        try:
            inspector = inspect(self.engine)
            all_tables = inspector.get_table_names()
            pattern = re.compile(r'^(.*)_30MinData$', re.IGNORECASE)
            tickers = [match.group(1).upper() for table in all_tables if (match := pattern.match(table))]
            return tickers
        except Exception as e:
            print(f"❌ Failed to inspect tables: {e}")
            return []
    def get_tickers_from_EOD_tables(self):
        """
        Extracts all tickers from tables that match the {ticker}_30MinData format.
        """
        try:
            inspector = inspect(self.engine)
            all_tables = inspector.get_table_names()
            pattern = re.compile(r'^(.*)_EOD_Data$', re.IGNORECASE)
            tickers = [match.group(1).upper() for table in all_tables if (match := pattern.match(table))]
            return tickers
        except Exception as e:
            print(f"❌ Failed to inspect tables: {e}")
            return []
        
    def get_tickers_from_Fundamentals_tables(self):
        """
        Extracts all tickers from tables that match the {ticker}_Fundamentals format.
        """
        try:
            inspector = inspect(self.engine)
            all_tables = inspector.get_table_names()
            pattern = re.compile(r'^(.*)_FundamentalsData$', re.IGNORECASE)
            tickers = [match.group(1).upper() for table in all_tables if (match := pattern.match(table))]
            return tickers
        except Exception as e:
            print(f"❌ Failed to inspect tables: {e}")
            return []
    
    def get_tickers_from_combined_tables(self):
        try:
            inspector = inspect(self.engine)
            all_tables = inspector.get_table_names()
            pattern = re.compile(r'^(.*)_CombinedData$', re.IGNORECASE)
            tickers = [match.group(1).upper() for table in all_tables if (match := pattern.match(table))]
            return tickers
        except Exception as e:
            print(f"❌ Failed to inspect tables: {e}")
            return []

In [3]:
pg = PostgresManager(
    host=os.getenv('host'),
    port=os.getenv('port'),
    dbname=os.getenv('dbname'),
    user=os.getenv('user'),
    password=os.getenv('password')
)

✅ PostgreSQL connection initialized.


In [4]:
eodTickers = pg.get_tickers_from_EOD_tables()

def rollingSharpe(df: pd.DataFrame, window: int = 90) -> pd.Series:
    """
    Calculate the rolling Sharpe ratio for a given DataFrame.
    Assumes the DataFrame has a 'close' column.
    """
    if 'close' not in df.columns:
        raise ValueError("DataFrame must contain a 'close' column.")
    
    # Calculate daily returns
    returns = df['close'].pct_change().dropna()
    
    # Calculate rolling mean and standard deviation
    rolling_mean = returns.rolling(window=window).mean()
    rolling_std = returns.rolling(window=window).std()
    
    # Calculate Sharpe ratio (assuming risk-free rate is 0)
    sharpe_ratio = rolling_mean / rolling_std
    
    return sharpe_ratio

def rollingVolatility(df: pd.DataFrame, window: int = 90) -> pd.Series:
    """
    Calculate the rolling volatility for a given DataFrame.
    Assumes the DataFrame has a 'close' column.
    """
    if 'close' not in df.columns:
        raise ValueError("DataFrame must contain a 'close' column.")
    
    # Calculate daily returns
    returns = df['close'].pct_change().dropna()
    
    # Calculate rolling standard deviation
    rolling_std = returns.rolling(window=window).std()
    
    return rolling_std

eodData = pd.DataFrame()
for ticker in eodTickers:
    df = pg.getTickerEODData(ticker)
    if not df.empty:
        #getting the close column of the dataframe
        #get calculate the return of the closed column of the dataframe
        #calculate the rolling 90 day sharpe of the close column of the dataframe
        #calculate the rolling 90 day volatility of the close column of the dataframe
        #add it to the eodData dataframe with the columns
        # {ticker}_close, {ticker}_sharpe, {ticker}_volatility
        eodData[f"{ticker}_close"] = df['close']
        eodData[f"{ticker}_return"] = df['close'].pct_change()
        eodData[f"{ticker}_sharpe"] = rollingSharpe(df)
        eodData[f"{ticker}_volatility"] = rollingVolatility(df)
        
    else:
        print(f"No data found for ticker: {ticker}")
        
eodData.fillna(0, inplace=True)
eodData.head(20)

✅ Retrieved 7181 rows of EOD data for CVS
✅ Retrieved 11213 rows of EOD data for AAPL
✅ Retrieved 9886 rows of EOD data for MSFT
✅ Retrieved 6635 rows of EOD data for NVDA
✅ Retrieved 7060 rows of EOD data for AMZN
✅ Retrieved 5234 rows of EOD data for GOOGL
✅ Retrieved 5234 rows of EOD data for GOOG
✅ Retrieved 3282 rows of EOD data for META
✅ Retrieved 3984 rows of EOD data for AVGO
✅ Retrieved 7317 rows of EOD data for BRK-B
✅ Retrieved 11400 rows of EOD data for BRK-A
✅ Retrieved 3759 rows of EOD data for TSLA
✅ Retrieved 13307 rows of EOD data for WMT
✅ Retrieved 11400 rows of EOD data for JPM
✅ Retrieved 13391 rows of EOD data for LLY
✅ Retrieved 4333 rows of EOD data for V
✅ Retrieved 4789 rows of EOD data for MA
✅ Retrieved 5798 rows of EOD data for NFLX
✅ Retrieved 9887 rows of EOD data for ORCL
✅ Retrieved 9805 rows of EOD data for COST
✅ Retrieved 11950 rows of EOD data for XOM
✅ Retrieved 11950 rows of EOD data for PG
✅ Retrieved 14003 rows of EOD data for JNJ
✅ Retrieved 1

/tmp/ipykernel_43696/3810752745.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eodData[f"{ticker}_close"] = df['close']
/tmp/ipykernel_43696/3810752745.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eodData[f"{ticker}_return"] = df['close'].pct_change()
/tmp/ipykernel_43696/3810752745.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-

✅ Retrieved 10240 rows of EOD data for UNH
✅ Retrieved 15966 rows of EOD data for GE
✅ Retrieved 5274 rows of EOD data for CRM
✅ Retrieved 8891 rows of EOD data for CSCO


/tmp/ipykernel_43696/3810752745.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eodData[f"{ticker}_close"] = df['close']
/tmp/ipykernel_43696/3810752745.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eodData[f"{ticker}_return"] = df['close'].pct_change()
/tmp/ipykernel_43696/3810752745.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-

✅ Retrieved 15991 rows of EOD data for IBM
✅ Retrieved 13391 rows of EOD data for WFC
✅ Retrieved 15991 rows of EOD data for CVX
✅ Retrieved 11400 rows of EOD data for ABT
✅ Retrieved 14855 rows of EOD data for MCD
✅ Retrieved 8116 rows of EOD data for INTU
✅ Retrieved 3253 rows of EOD data for NOW


...

✅ Retrieved 4161 rows of EOD data for LOPE
✅ Retrieved 2400 rows of EOD data for AXSM
✅ Retrieved 7950 rows of EOD data for ITRI
✅ Retrieved 3767 rows of EOD data for CHRD
✅ Retrieved 6943 rows of EOD data for GPI
✅ Retrieved 2281 rows of EOD data for SITE
✅ Retrieved 176 rows of EOD data for AMTM


/tmp/ipykernel_43696/3810752745.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eodData[f"{ticker}_close"] = df['close']
/tmp/ipykernel_43696/3810752745.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eodData[f"{ticker}_return"] = df['close'].pct_change()
/tmp/ipykernel_43696/3810752745.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-

✅ Retrieved 11400 rows of EOD data for TFX
✅ Retrieved 9830 rows of EOD data for BCPC
✅ Retrieved 2609 rows of EOD data for BOX
✅ Retrieved 3806 rows of EOD data for SPSC
✅ Retrieved 1258 rows of EOD data for LEGN
✅ Retrieved 743 rows of EOD data for IPX


/tmp/ipykernel_43696/3810752745.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eodData[f"{ticker}_close"] = df['close']
/tmp/ipykernel_43696/3810752745.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  eodData[f"{ticker}_return"] = df['close'].pct_change()
/tmp/ipykernel_43696/3810752745.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-

,CVS_close,CVS_return,CVS_sharpe,CVS_volatility,AAPL_close,AAPL_return,AAPL_sharpe,AAPL_volatility,MSFT_close,MSFT_return,...,SPSC_sharpe,SPSC_volatility,LEGN_close,LEGN_return,LEGN_sharpe,LEGN_volatility,IPX_close,IPX_return,IPX_sharpe,IPX_volatility
date,,,,,,,,,,,,,,,,,,,,,
1996-11-20,10.06,0.000000,0.0,0.0,0.22321,0.004998,0.135267,0.037039,9.58,-0.016427,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-11-21,10.06,0.000000,0.0,0.0,0.21875,-0.019981,0.128959,0.037129,9.40,-0.018789,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-11-22,10.38,0.031809,0.0,0.0,0.22545,0.030629,0.089633,0.027836,9.41,0.001064,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-11-25,10.50,0.011561,0.0,0.0,0.22321,-0.009936,0.087992,0.027853,9.59,0.019129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-11-26,10.38,-0.011429,0.0,0.0,0.21652,-0.029972,0.085447,0.027922,9.61,0.002086,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-11-27,10.31,-0.006744,0.0,0.0,0.21875,0.010299,0.084638,0.027914,9.72,0.011446,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-11-29,10.28,-0.002910,0.0,0.0,0.21540,-0.015314,0.072404,0.027942,9.80,0.008230,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-12-02,10.28,0.000000,0.0,0.0,0.22433,0.041458,0.084413,0.028241,9.86,0.006122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-12-03,10.19,-0.008755,0.0,0.0,0.22433,0.000000,0.066652,0.027827,9.67,-0.019270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#saving the eodData dataframe to a csv file
eodData.to_csv('eodData.csv', index=True)


In [8]:
#trim the EOD data columns to only include 10% of the tickers
#randomly remove the {ticker}_close, {ticker}_sharpe, {ticker}_return {ticker}_volatility columns
import numpy as np

def trim_eod_data(df: pd.DataFrame, percentage: float = 0.1) -> pd.DataFrame:
    """
    Randomly removes columns from the DataFrame based on the specified percentage.
    """
    if percentage < 0 or percentage > 1:
        raise ValueError("Percentage must be between 0 and 1.")
    
    # Get all columns that match the pattern
    ticker_columns = [col for col in df.columns if re.match(r'^[A-Z]+_(close|sharpe|return|volatility)$', col)]
    
    # Calculate number of columns to drop
    num_to_drop = int(len(ticker_columns) * percentage)
    
    # Randomly select columns to drop
    columns_to_drop = np.random.choice(ticker_columns, size=num_to_drop, replace=False)
    
    # Drop the selected columns
    df_trimmed = df.drop(columns=columns_to_drop)
    
    return df_trimmed
# Trim the EOD data to only include 10% of the tickers
trimmed_eod_data = trim_eod_data(eodData, percentage=0.1)


In [10]:
trimmed_eod_data.head(20)

,CVS_close,CVS_return,CVS_sharpe,CVS_volatility,AAPL_close,AAPL_return,AAPL_volatility,MSFT_close,MSFT_return,MSFT_sharpe,MSFT_volatility,NVDA_close,NVDA_return,NVDA_sharpe,NVDA_volatility,AMZN_close,AMZN_return,AMZN_sharpe,AMZN_volatility,GOOGL_close,GOOGL_return,GOOGL_sharpe,GOOGL_volatility,GOOG_return,GOOG_sharpe,GOOG_volatility,META_close,META_return,META_sharpe,META_volatility,AVGO_close,AVGO_return,AVGO_sharpe,AVGO_volatility,BRK-B_close,BRK-B_return,BRK-B_sharpe,BRK-B_volatility,BRK-A_close,BRK-A_return,BRK-A_sharpe,BRK-A_volatility,TSLA_close,TSLA_return,TSLA_sharpe,TSLA_volatility,WMT_close,WMT_return,WMT_sharpe,JPM_close,...,WSM_position,WSO_weight,WSO_position,WST_weight,WST_position,WTFC_weight,WTFC_position,WTRG_weight,WTRG_position,WTS_weight,WTS_position,WWD_weight,WWD_position,WY_weight,WY_position,WYNN_weight,WYNN_position,X_weight,X_position,XEL_weight,XEL_position,XOM_weight,XOM_position,XPO_weight,XPO_position,XYL_weight,XYL_position,XYZ_weight,XYZ_position,YUM_weight,YUM_position,Z_weight,Z_position,ZBH_weight,ZBH_position,ZBRA_weight,ZBRA_position,ZG_weight,ZG_position,ZION_weight,ZION_position,ZM_weight,ZM_position,ZS_weight,ZS_position,ZTS_weight,ZTS_position,ZWS_weight,ZWS_position,portfolio_return
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1996-11-20,10.06,0.000000,0.0,0.0,0.22321,0.004998,0.037039,9.58,-0.016427,0.212375,0.015451,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.120001,-0.005396,0.083682,0.009322,33400.0,0.000000,0.080889,0.009963,0.0,0.0,0.0,0.0,4.15,0.017157,0.060224,30.75,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1996-11-21,10.06,0.000000,0.0,0.0,0.21875,-0.019981,0.037129,9.40,-0.018789,0.186446,0.015571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.100000,-0.000904,0.060950,0.009112,33200.0,-0.005988,0.048814,0.009675,0.0,0.0,0.0,0.0,4.21,0.014458,0.034794,30.25,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1996-11-22,10.38,0.031809,0.0,0.0,0.22545,0.030629,0.027836,9.41,0.001064,0.172298,0.015420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.160000,0.002715,0.058532,0.009104,33200.0,0.000000,0.039959,0.009643,0.0,0.0,0.0,0.0,4.25,0.009501,0.043884,30.50,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1996-11-25,10.50,0.011561,0.0,0.0,0.22321,-0.009936,0.027853,9.59,0.019129,0.178399,0.015502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.020000,-0.006318,0.056504,0.009116,33200.0,0.000000,0.054824,0.009546,0.0,0.0,0.0,0.0,4.25,0.000000,0.047051,31.79,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1996-11-26,10.38,-0.011429,0.0,0.0,0.21652,-0.029972,0.027922,9.61,0.002086,0.188270,0.015437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.000000,-0.000908,0.071212,0.009014,32700.0,-0.015060,0.047664,0.009632,0.0,0.0,0.0,0.0,4.23,-0.004706,0.065914,31.38,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1996-11-27,10.31,-0.006744,0.0,0.0,0.21875,0.010299,0.027914,9.72,0.011446,0.270814,0.013779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.000000,0.000000,0.100619,0.008711,33200.0,0.015291,0.084113,0.009595,0.0,0.0,0.0,0.0,4.23,0.000000,0.069294,31.33,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,

In [14]:
import numpy as np
import pandas as pd
import cvxpy as cp
from tqdm import tqdm

def optimize_portfolio_strategy(df, rebalance_freq=60, corr_window=90, sharpe_window=90, top_pairs=10, lambda_risk=0.5):
    """
    Optimize portfolio based on rolling Sharpe ratios and correlation pairs
    
    Parameters:
    df : DataFrame
        Input data with columns in format {ticker}_{metric}
    rebalance_freq : int
        Days between rebalancing (default: 60)
    corr_window : int
        Window size for correlation calculation (default: 90)
    sharpe_window : int
        Window size for Sharpe ratio calculation (default: 90)
    top_pairs : int
        Number of low-correlation pairs to consider (default: 10)
    lambda_risk : float
        Risk aversion parameter for optimization (default: 0.5)
    """
    # Extract tickers from column names
    tickers = list(set(col.split('_')[0] for col in df.columns if '_close' in col))
    tickers.sort()
    
    # Create output columns
    for t in tickers:
        df[f'{t}_weight'] = 0.0
        df[f'{t}_position'] = 0
    
    # Add portfolio return column
    df['portfolio_return'] = 0.0
    
    # Identify rebalance dates
    valid_dates = df.index[sharpe_window - 1:-1]
    rebalance_dates = valid_dates[::rebalance_freq]
    
    for i in tqdm(range(len(rebalance_dates))):
        rebalance_date = rebalance_dates[i]
        idx = df.index.get_loc(rebalance_date)
        
        # Get data windows
        corr_start = max(0, idx - corr_window + 1)
        sharpe_start = max(0, idx - sharpe_window + 1)
        
        corr_window_data = df.iloc[corr_start:idx+1]
        sharpe_window_data = df.iloc[sharpe_start:idx+1]
        
        # Calculate rolling Sharpe ratios
        sharpes = {}
        for t in tickers:
            returns = sharpe_window_data[f'{t}_return']
            if returns.notnull().sum() < 2:
                sharpes[t] = 0
                continue
                
            excess_returns = returns - returns.mean()
            sharpe = np.sqrt(sharpe_window) * returns.mean() / returns.std()
            sharpes[t] = sharpe if not np.isnan(sharpe) else 0
        
        # Calculate correlation matrix
        corr_matrix = pd.DataFrame(index=tickers, columns=tickers)
        for t1 in tickers:
            for t2 in tickers:
                if t1 == t2:
                    corr_matrix.loc[t1, t2] = 1.0
                    continue
                    
                closes1 = corr_window_data[f'{t1}_close'].dropna()
                closes2 = corr_window_data[f'{t2}_close'].dropna()
                common_idx = closes1.index.intersection(closes2.index)
                
                if len(common_idx) < 2:
                    corr_matrix.loc[t1, t2] = 0
                    continue
                    
                corr = np.corrcoef(closes1.loc[common_idx], closes2.loc[common_idx])[0, 1]
                corr_matrix.loc[t1, t2] = corr if not np.isnan(corr) else 0
        
        # Find low-correlation pairs
        pairs = []
        for i1, t1 in enumerate(tickers):
            for i2, t2 in enumerate(tickers[i1+1:], i1+1):
                pairs.append((t1, t2, corr_matrix.loc[t1, t2]))
        
        pairs_sorted = sorted(pairs, key=lambda x: x[2])[:top_pairs]
        
        # Prepare candidate assets for optimization
        candidate_returns = []   # List to store return series for all candidates
        candidate_info = []      # Metadata for each candidate
        candidate_assets = set() # Track assets already included
        used_assets = set()      # Assets already used in candidates
        
        # 1. Process pairs according to Sharpe comparison rules
        for t1, t2, _ in pairs_sorted:
            if t1 in used_assets or t2 in used_assets:
                continue
                
            # Get returns for both assets
            ret1 = sharpe_window_data[f'{t1}_return'].dropna()
            ret2 = sharpe_window_data[f'{t2}_return'].dropna()
            common_idx = ret1.index.intersection(ret2.index)
            
            if len(common_idx) < 5:  # Minimum data requirement
                continue
                
            # Determine positions based on individual Sharpes
            pos1 = 1 if sharpes[t1] >= 0 else -1
            pos2 = 1 if sharpes[t2] >= 0 else -1
            
            # Calculate individual and pair returns
            ret1_common = pos1 * ret1.loc[common_idx]
            ret2_common = pos2 * ret2.loc[common_idx]
            pair_ret = 0.5 * (ret1_common + ret2_common)
            
            # Calculate Sharpe ratios for comparison
            def calc_sharpe(returns):
                if len(returns) < 2 or returns.std() == 0:
                    return 0
                return np.sqrt(len(returns)) * returns.mean() / returns.std()
                
            sharpe_pair = calc_sharpe(pair_ret)
            sharpe_t1 = calc_sharpe(ret1_common)
            sharpe_t2 = calc_sharpe(ret2_common)
            
            # Apply selection rules based on Sharpe comparison
            if sharpe_pair > sharpe_t1 and sharpe_pair > sharpe_t2:
                # Use the pair
                candidate_returns.append(pair_ret)
                candidate_info.append(('pair', t1, t2, pos1, pos2))
                used_assets.update([t1, t2])
                candidate_assets.update([t1, t2])
            elif sharpe_t1 > sharpe_t2 and sharpe_t1 > sharpe_pair:
                # Use only t1
                candidate_returns.append(ret1_common)
                candidate_info.append(('individual', t1, pos1))
                used_assets.add(t1)
                candidate_assets.add(t1)
            elif sharpe_t2 > sharpe_t1 and sharpe_t2 > sharpe_pair:
                # Use only t2
                candidate_returns.append(ret2_common)
                candidate_info.append(('individual', t2, pos2))
                used_assets.add(t2)
                candidate_assets.add(t2)
        
        # 2. Add top individual tickers not already included
        # Filter to assets not used in any candidate
        available_tickers = [t for t in tickers if t not in used_assets]
        
        # Get top Sharpes from available tickers
        top_individuals = sorted(
            [(t, sharpes[t]) for t in available_tickers],
            key=lambda x: x[1],
            reverse=True
        )[:top_pairs]
        
        for t, _ in top_individuals:
            # Determine position based on Sharpe
            pos = 1 if sharpes[t] >= 0 else -1
            
            # Get returns
            returns = sharpe_window_data[f'{t}_return'].dropna()
            if len(returns) < 5:
                continue
                
            candidate_returns.append(pos * returns)
            candidate_info.append(('individual', t, pos))
            candidate_assets.add(t)
        
        # Proceed only if we have at least 2 candidates
        if len(candidate_returns) < 2:
            continue
            
        # Align return series to common index
        return_df = pd.DataFrame(index=sharpe_window_data.index)
        for i, ret_series in enumerate(candidate_returns):
            return_df[f'candidate_{i}'] = ret_series
            
        return_df = return_df.dropna()
        
        if len(return_df) < 5:
            continue
            
        # Prepare for optimization
        R = return_df.values
        mu = np.mean(R, axis=0)
        Sigma = np.cov(R, rowvar=False)
        
        # CVXPY optimization
        w = cp.Variable(len(mu))
        expected_return = mu.T @ w
        risk = cp.quad_form(w, Sigma)
        
        problem = cp.Problem(
            cp.Maximize(expected_return - lambda_risk * risk),
            [cp.sum(w) == 1, w >= 0]
        )
        problem.solve()
        
        if w.value is None:
            continue
            
        # Allocate weights to underlying assets
        weights = w.value
        asset_weights = {t: 0.0 for t in tickers}
        
        for i, info in enumerate(candidate_info):
            wt = weights[i]
            if info[0] == 'pair':
                _, t1, t2, pos1, pos2 = info
                asset_weights[t1] += 0.5 * wt * pos1
                asset_weights[t2] += 0.5 * wt * pos2
            else:  # individual
                _, t, pos = info
                asset_weights[t] += wt * pos
        
        # Store weights and positions
        for t in tickers:
            weight_val = asset_weights[t]
            df.at[rebalance_date, f'{t}_weight'] = weight_val
            df.at[rebalance_date, f'{t}_position'] = np.sign(weight_val) if weight_val != 0 else 0
    
    # Forward-fill weights between rebalance dates
    for t in tickers:
        weight_col = f'{t}_weight'
        position_col = f'{t}_position'
        
        # Forward fill with rebalancing constraint
        current_weight = np.nan
        current_position = np.nan
        
        for i in range(len(df)):
            if not np.isnan(df.iloc[i][weight_col]):
                current_weight = df.iloc[i][weight_col]
                current_position = df.iloc[i][position_col]
            else:
                if i > 0 and (i % rebalance_freq) == 0:
                    # Reset at rebalance points if not explicitly set
                    current_weight = 0
                    current_position = 0
                
                df.iat[i, df.columns.get_loc(weight_col)] = current_weight
                df.iat[i, df.columns.get_loc(position_col)] = current_position
    
    # Calculate portfolio returns
    for t in tickers:
        df['portfolio_return'] += df[f'{t}_weight'] * df[f'{t}_return']
    
    return df


# Run the optimization strategy on the trimmed EOD data
optimized_portfolio = optimize_portfolio_strategy(eodData, rebalance_freq=60, corr_window=90, sharpe_window=90, top_pairs=10, lambda_risk=0.5)

  0%|          | 0/119 [00:00<?, ?it/s]/tmp/ipykernel_43696/3456235786.py:60: RuntimeWarning: invalid value encountered in scalar divide
  sharpe = np.sqrt(sharpe_window) * returns.mean() / returns.std()
/home/mashroor10/PMProject/PorfolioManager/.venv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/mashroor10/PMProject/PorfolioManager/.venv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
  1%|          | 1/119 [10:03<19:47:05, 603.61s/it]/tmp/ipykernel_43696/3456235786.py:60: RuntimeWarning: invalid value encountered in scalar divide
  sharpe = np.sqrt(sharpe_window) * returns.mean() / returns.std()
/home/mashroor10/PMProject/PorfolioManager/.venv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/hom

KeyboardInterrupt: 

In [ ]:
import numpy as np
import cvxpy as cp
import cupy as cp_gpu
from numba import cuda, jit
from tqdm import tqdm

# Initialize GPU context
cuda.select_device(0)
cp_gpu.cuda.Device(0).use()

# GPU-accelerated Sharpe ratio calculation
@cuda.jit
def gpu_sharpe(returns, output, window_size):
    i = cuda.grid(1)
    if i < returns.shape[0]:
        if returns.shape[1] < 2:
            output[i] = 0.0
            return

        mean = 0.0
        for j in range(returns.shape[1]):
            mean += returns[i, j]
        mean /= returns.shape[1]

        std = 0.0
        for j in range(returns.shape[1]):
            diff = returns[i, j] - mean
            std += diff * diff
        std = (std / (returns.shape[1] - 1)) ** 0.5

        if std == 0:
            output[i] = 0.0
        else:
            output[i] = (mean / std) * (window_size ** 0.5)

def optimize_portfolio_strategy_gpu(df, rebalance_freq=60, corr_window=90,
                                   sharpe_window=90, top_pairs=10, lambda_risk=0.5):
    # Extract tickers from column names
    tickers = list(set(col.split('_')[0] for col in df.columns if '_close' in col))
    tickers.sort()

    # Create output columns
    for t in tickers:
        df[f'{t}_weight'] = 0.0
        df[f'{t}_position'] = 0

    # Add portfolio return column
    df['portfolio_return'] = 0.0

    # Identify rebalance dates
    valid_dates = df.index[sharpe_window - 1:-1]
    rebalance_dates = valid_dates[::rebalance_freq]

    # Convert main DataFrame to GPU DataFrame
    gdf = cudf.DataFrame.from_pandas(df)

    for i in tqdm(range(len(rebalance_dates))):
        rebalance_date = rebalance_dates[i]
        idx = df.index.get_loc(rebalance_date)

        # Get data windows
        corr_start = max(0, idx - corr_window + 1)
        sharpe_start = max(0, idx - sharpe_window + 1)

        corr_window_data = gdf.iloc[corr_start:idx+1]
        sharpe_window_data = gdf.iloc[sharpe_start:idx+1]

        # GPU: Calculate rolling Sharpe ratios
        sharpes = {}
        sharpe_returns = cudf.DataFrame()

        for t in tickers:
            col_name = f'{t}_return'
            if col_name in sharpe_window_data.columns:
                sharpe_returns[t] = sharpe_window_data[col_name].fillna(0)

        # Transfer to CuPy for GPU computation
        returns_array = sharpe_returns.to_cupy().T

        # Allocate GPU memory for Sharpe results
        sharpe_results = cp_gpu.zeros(returns_array.shape[0])

        # Configure and launch kernel
        threads_per_block = 64
        blocks_per_grid = (returns_array.shape[0] + threads_per_block - 1) // threads_per_block
        gpu_sharpe[blocks_per_grid, threads_per_block](returns_array, sharpe_results, sharpe_window)

        # Copy results back to CPU
        cpu_sharpes = sharpe_results.get()
        for j, t in enumerate(sharpe_returns.columns):
            sharpes[t] = cpu_sharpes[j]

        # GPU: Calculate correlation matrix using cuDF
        corr_matrix = cudf.DataFrame()

        # Create returns matrix for correlation period
        corr_returns = cudf.DataFrame()
        for t in tickers:
            col_name = f'{t}_return'
            if col_name in corr_window_data.columns:
                corr_returns[t] = corr_window_data[col_name].fillna(0)

        # Compute correlation matrix on GPU
        gpu_corr = corr_returns.corr(method='pearson').fillna(0)

        # Convert to pandas for easier processing (small matrix)
        corr_matrix = gpu_corr.to_pandas()

        # Find low-correlation pairs
        pairs = []
        for i1, t1 in enumerate(tickers):
            for i2, t2 in enumerate(tickers[i1+1:], i1+1):
                if t1 in corr_matrix.index and t2 in corr_matrix.columns:
                    corr_val = corr_matrix.at[t1, t2]
                    pairs.append((t1, t2, corr_val))

        pairs_sorted = sorted(pairs, key=lambda x: x[2])[:top_pairs]

        # Prepare candidate assets for optimization
        candidate_returns = []   # List to store return series for all candidates
        candidate_info = []      # Metadata for each candidate
        candidate_assets = set() # Track assets already included
        used_assets = set()      # Assets already used in candidates

        # 1. Process pairs according to Sharpe comparison rules
        for t1, t2, _ in pairs_sorted:
            if t1 in used_assets or t2 in used_assets:
                continue

            # Get returns for both assets
            ret1 = sharpe_window_data[f'{t1}_return'].fillna(0).to_cupy()
            ret2 = sharpe_window_data[f'{t2}_return'].fillna(0).to_cupy()

            # Determine positions based on individual Sharpes
            pos1 = 1 if sharpes.get(t1, 0) >= 0 else -1
            pos2 = 1 if sharpes.get(t2, 0) >= 0 else -1

            # Calculate individual and pair returns
            ret1_common = pos1 * ret1
            ret2_common = pos2 * ret2
            pair_ret = 0.5 * (ret1_common + ret2_common)

            # Calculate Sharpe ratios for comparison
            def calc_sharpe(returns):
                if returns.size < 2 or returns.std() == 0:
                    return 0
                return returns.mean() / returns.std() * (returns.size ** 0.5)

            sharpe_pair = calc_sharpe(pair_ret)
            sharpe_t1 = calc_sharpe(ret1_common)
            sharpe_t2 = calc_sharpe(ret2_common)

            # Apply selection rules
            if sharpe_pair > sharpe_t1 and sharpe_pair > sharpe_t2:
                candidate_returns.append(pair_ret.get())
                candidate_info.append(('pair', t1, t2, pos1, pos2))
                used_assets.update([t1, t2])
                candidate_assets.update([t1, t2])
            elif sharpe_t1 > sharpe_t2 and sharpe_t1 > sharpe_pair:
                candidate_returns.append(ret1_common.get())
                candidate_info.append(('individual', t1, pos1))
                used_assets.add(t1)
                candidate_assets.add(t1)
            elif sharpe_t2 > sharpe_t1 and sharpe_t2 > sharpe_pair:
                candidate_returns.append(ret2_common.get())
                candidate_info.append(('individual', t2, pos2))
                used_assets.add(t2)
                candidate_assets.add(t2)

        # 2. Add top individual tickers not already included
        available_tickers = [t for t in tickers if t not in used_assets]

        # Get top Sharpes from available tickers
        top_individuals = sorted(
            [(t, sharpes.get(t, 0)) for t in available_tickers],
            key=lambda x: x[1],
            reverse=True
        )[:top_pairs]

        for t, _ in top_individuals:
            pos = 1 if sharpes.get(t, 0) >= 0 else -1
            returns = sharpe_window_data[f'{t}_return'].fillna(0).to_cupy()

            if returns.size < 5:
                continue

            candidate_returns.append((pos * returns).get())
            candidate_info.append(('individual', t, pos))
            candidate_assets.add(t)

        # Proceed only if we have at least 2 candidates
        if len(candidate_returns) < 2:
            continue

        # Align return series to common length
        min_length = min(len(r) for r in candidate_returns)
        aligned_returns = np.array([r[-min_length:] for r in candidate_returns]).T

        # GPU: Compute mean and covariance on GPU
        cupy_returns = cp_gpu.array(aligned_returns)
        mu = cp_gpu.mean(cupy_returns, axis=0).get()
        Sigma = cp_gpu.cov(cupy_returns, rowvar=False).get()

        # CVXPY optimization (still on CPU)
        w = cp.Variable(len(mu))
        expected_return = mu.T @ w
        risk = cp.quad_form(w, Sigma)

        problem = cp.Problem(
            cp.Maximize(expected_return - lambda_risk * risk),
            [cp.sum(w) == 1, w >= 0]
        )
        problem.solve()

        if w.value is None:
            continue

        # Allocate weights to underlying assets
        weights = w.value
        asset_weights = {t: 0.0 for t in tickers}

        for i, info in enumerate(candidate_info):
            wt = weights[i]
            if info[0] == 'pair':
                _, t1, t2, pos1, pos2 = info
                asset_weights[t1] += 0.5 * wt * pos1
                asset_weights[t2] += 0.5 * wt * pos2
            else:  # individual
                _, t, pos = info
                asset_weights[t] += wt * pos

        # Store weights and positions
        for t in tickers:
            weight_val = asset_weights[t]
            df.at[rebalance_date, f'{t}_weight'] = weight_val
            df.at[rebalance_date, f'{t}_position'] = np.sign(weight_val) if weight_val != 0 else 0

    # Forward-fill weights between rebalance dates
    # Create rebalance mask
    rebalance_mask = pd.Series(False, index=df.index)
    rebalance_mask.loc[rebalance_dates] = True

    print("\n[PHASE 5] Vectorized weight and position filling")

    # Vectorized forward-filling with reset logic
    for t in tqdm(tickers, desc="Processing tickers", unit="ticker"):
        weight_col = f'{t}_weight'
        position_col = f'{t}_position'

        # Create a mask for reset points (rebalance dates with NaN values)
        reset_mask = rebalance_mask & df[weight_col].isna()

        # Forward fill with reset at specified rebalance points
        df[weight_col] = df[weight_col].ffill()
        df[position_col] = df[position_col].ffill()

        # Apply reset logic
        df.loc[reset_mask, weight_col] = 0
        df.loc[reset_mask, position_col] = 0

    print("\n[PHASE 6] Vectorized portfolio return calculation")

    # Vectorized portfolio return calculation
    weight_cols = [f'{t}_weight' for t in tickers]
    return_cols = [f'{t}_return' for t in tickers]

    # Calculate portfolio returns in one vectorized operation
    df['portfolio_return'] = (df[weight_cols].values * df[return_cols].values).sum(axis=1)

    return df